In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyarrow.parquet as pq
import matplotlib.pyplot as plt

In [2]:
table = pq.read_table("/kaggle/input/ml4sci/QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet"  )
table

pyarrow.Table
X_jets: list<item: list<item: list<item: double>>>
  child 0, item: list<item: list<item: double>>
      child 0, item: list<item: double>
          child 0, item: double
pt: double
m0: double
y: double
----
X_jets: [[[[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0]],[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0,0,0,0,0,...,0,0,0,0.09783933311700821,0.09783933311700821],[0,0,0,0,0,...,0,0,0,0,0]],[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0]]]],[[[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0]],[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0]],[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0]]]],...,[[[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0

In [3]:
images_data = table['X_jets']

labels = table['y'].to_numpy()

mass = table["m0"].to_numpy()
pt = table["pt"].to_numpy()

del table

labels.shape, mass.shape, pt.shape

((55494,), (55494,), (55494,))

In [5]:
import os
os.mkdir('/kaggle/working/run2/')

In [ ]:
count = 1
batch_size = 1000 # Full Size
images_set = np.empty((batch_size,125,125,3))
counter = 0
import h5py

for image_pq_array in images_data.iterchunks():
    #for j in image_pq_array:
        # print(( j.values.to_pandas().shape ))
    # print(image_pq_array[0].values.to_pandas().shape)
    image = np.moveaxis( np.array( [ np.array( [x for x in image_pq_array[0].values.to_pandas()[i]] ) for i in range(3) ] ), 0, -1)
    # print(count-1)
    images_set[count-1-batch_size*counter] = image
    
    if count % batch_size == 0 or count==len(labels):
        print( "----------------------------------------------------------" )
        # print(count-1-batch_size*counter )
        print(images_set.shape, count-batch_size, count-1 )
        with h5py.File("/kaggle/working/run2/images_data_run2_" + str(count) + ".h5", 'w') as hf:
            hf.create_dataset('images_data', data=images_set,compression='gzip')
            hf.create_dataset('labels', data=labels[count-batch_size:count-1] )
            hf.create_dataset('mass', data=mass[count-batch_size:count-1] )
            hf.create_dataset('pt', data=pt[count-batch_size:count-1] )
        images_set = np.empty((batch_size,125,125,3))
        counter = counter + 1
        print( "----------------------------------------------------------" )
    count = count + 1

----------------------------------------------------------
(1000, 125, 125, 3) 0 999
----------------------------------------------------------
----------------------------------------------------------
(1000, 125, 125, 3) 1000 1999
----------------------------------------------------------
----------------------------------------------------------
(1000, 125, 125, 3) 2000 2999
----------------------------------------------------------
----------------------------------------------------------
(1000, 125, 125, 3) 3000 3999
----------------------------------------------------------
----------------------------------------------------------
(1000, 125, 125, 3) 4000 4999
----------------------------------------------------------
----------------------------------------------------------
(1000, 125, 125, 3) 5000 5999
----------------------------------------------------------
----------------------------------------------------------
(1000, 125, 125, 3) 6000 6999
---------------------------

In [ ]:
images_set.shape

In [ ]:
# One line code for above "for loop" | consumes more ram
# images_set = np.array( [  np.array( [ np.array( [x for x in image_array[0].values.to_pandas()[i]] ) for i in range(3) ] )  for image_array in  table['X_jets'].iterchunks() ] )

In [ ]:
import matplotlib.pyplot as plt

img = images_set[10000]
c = plt.imshow(img)
plt.colorbar(c)
plt.title('Example Image')
plt.show()

In [ ]:
# import h5py
# with h5py.File("data_run2.h5", 'w') as hf:
#     hf.create_dataset('images_data', data=images_set,compression='gzip')
#     hf.create_dataset('labels', data=labels,compression='gzip')
#     hf.create_dataset('mass', data=mass,compression='gzip')
#     hf.create_dataset('pt', data=pt,compression='gzip')

***
***

# Using Pandas

In [ ]:
data = pd.read_parquet("/kaggle/input/google-summer-code-datasets/common task 2/QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet")
# data

In [ ]:
data_processed = np.stack([np.concatenate(temp[0]).reshape(125, 125, 1) for temp in data["X_jets"]], axis=2)
data_processed = np.moveaxis(data_processed[:,:,:,0], 2, 0)
pt_data = data['pt'].values
m_data = data['m0'].values
data_processed.shape

In [ ]:
del data

In [ ]:
import h5py
# Assuming you have a NumPy array called data_processed
with h5py.File('data_processed_QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.h5', 'w') as hf:
    hf.create_dataset('data', data=data_processed,compression='gzip')

with h5py.File('pt_data_QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.h5', 'w') as hf:
    hf.create_dataset('pt_data', data=pt_data)
with h5py.File('m_data_QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.h5', 'w') as hf:
    hf.create_dataset('m_data', data=m_data)

In [ ]:
# Open the HDF5 file
with h5py.File('/kaggle/working/data_processed.h5', 'r') as hf:
    # Access the compressed dataset
    data_loaded = hf['data'][()]

In [ ]:
data_loaded.shape

In [ ]:
# data_processed = np.array([])

# temp = data["X_jets"][0]
# yo = np.array([])
# for i in temp[0]:
#     yo = np.concatenate( (yo,i) )
# data_processed = yo.reshape(125,125,1)

# for row in tqdm(range(1,data.shape[0])):
#     temp = data["X_jets"][row]
#     yo = np.array([])
#     for i in temp[0]:
#         yo = np.concatenate( (yo,i) )
#     yo = yo.reshape(125,125,1)
    
#     data_processed = np.concatenate((data_processed,yo),axis=2)
    
# data_processed.shape

In [ ]:
from matplotlib import pyplot as plt
yo = data_processed[0]
plt.imshow(yo, interpolation='nearest')
plt.show()

In [ ]:
# Open the HDF5 file
import h5py
with h5py.File('/home/abhishek/triplet_codons/Google Summer Code Projects/Data/common task 1/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5', 'r') as hf:
    # Access the compressed dataset
    print( hf.keys() )
    X_data_loaded_1 = hf['X'][()]
    y_data_loaded_1 = hf['y'][()]

with h5py.File('/home/abhishek/triplet_codons/Google Summer Code Projects/Data/common task 1/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5', 'r') as hf:
    # Access the compressed dataset
    print( hf.keys() )
    X_data_loaded_2 = hf['X'][()]
    y_data_loaded_2 = hf['y'][()]

# with h5py.File('/home/abhishek/triplet_codons/Google Summer Code Projects/common_task_2_data_h5/pt_data_run2.h5', 'r') as hf:
#     # Access the compressed dataset
#     data_loaded_3 = hf['pt_data'][()]


X_data_loaded_1.shape, X_data_loaded_2.shape, y_data_loaded_1.shape, y_data_loaded_2.shape

In [ ]:
data_final = np.concatenate((X_data_loaded_1, X_data_loaded_1), axis=0)
data_final.shape

In [ ]:
# Assuming you have a NumPy array called data_processed
with h5py.File('X_data_task_1'+".h5", 'w') as hf:
    hf.create_dataset('X_data', data=data_final,compression='gzip')

In [ ]:
from pyarrow import parquet as pq
table = pq.read_table("/kaggle/input/google-summer-code-datasets/common task 2/QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet")

from pyarrow import csv
csv.write_csv(data=table,output_file="run1.csv")

In [ ]:
# Open the HDF5 file
import h5py
with h5py.File('/kaggle/input/google-summer-code-datasets/common_task_1_data_h5/X_data_task_1.h5', 'r') as hf:
    # Access the compressed dataset
    data_loaded = hf['X_data'][()]

In [ ]:
data_loaded.shape

In [ ]:
data_loaded[1].shape